In [6]:
import json
import requests

import pandas as pd 

In [12]:
def json_to_df(data):
    elements = data['elements']
    places = {'tipo': [], 'lat': [], 'lon': [], 'name': [], 'address': []}
    
    for i in elements:
        
        tipo = i.get('tags', None).get('amenity', None)
        latitude = i.get('lat', None)
        longitude = i.get('lon', None)
        name = i.get('tags', {}).get('name', "NO NAME")
        street = i.get('tags', {}).get('addr:street', "NO STREET")
        number = i.get('tags', {}).get('addr:housenumber', 9999)

        places['tipo'].append(tipo)
        places['lat'].append(latitude)
        places['lon'].append(longitude)
        places['name'].append(name)
        places['address'].append(street + ' ' + str(number))

            
    return pd.DataFrame(places)

In [61]:
tipo = ["restaurant", "bar", "pub", "cafe", "cinema", "theatre"]
barcelona = (41.33,2.06,41.46,2.27)
madrid = (40.256,-3.995,40.543,-3.306)
bilbao = (43.199,-3.069,43.373,-2.828)
valencia = (39.408,-0.4758,39.548,-0.273)
malaga = (36.658,-4.501,36.754,-4.351)
sevilla = (37.332,-6.094,37.439,-5.893)

In [25]:
#Función de prueba, no se utiliza
def tablas_(iterator, location):

    for ii, k in enumerate(iterator):
        try:
            overpass_url = "http://overpass-api.de/api/interpreter"
            overpass_query = f"""
            [out:json];
            node["amenity"={k}]
              {location};
            out;
            """
            
            response = requests.get(overpass_url, 
                                    params={'data': overpass_query})
            data = response.json()
            

            if ii==0:
                df = json_to_df(data)
                print(f"Iteración{ii}:{df.shape}\n---------------------------\n")
            else:
                df = pd.concat([df, json_to_df(data)])
                print(f"Iteración{ii}:{df.shape}\n---------------------------\n")
        except:

            print(f"No hay data para {k} en las coordenadas {location}\n---------------------------\n")
        

    return df

In [26]:
#Ejecución de función en desuso
df_barcelona = tablas_(tipo, barcelona)

Iteración0:(3142, 5)
---------------------------

Iteración1:(4117, 5)
---------------------------

No hay data para pub en las coordenadas (41.33, 2.06, 41.46, 2.27)
---------------------------

No hay data para fast-food en las coordenadas (41.33, 2.06, 41.46, 2.27)
---------------------------

No hay data para cinema en las coordenadas (41.33, 2.06, 41.46, 2.27)
---------------------------

Iteración5:(4168, 5)
---------------------------



In [65]:
def tablas(nodes, location, col):
    """Con esta función se obtienen los locales para los nodes definidos en la localización (marcada por 4 puntos), que traen los argumentos."""

    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = f"""
    [out:json];
    node["amenity"]{location};
    out;
    """
            
    response = requests.get(overpass_url, 
                            params={'data': overpass_query})
    data = response.json() 
    df_total = json_to_df(data)
    
    for ii,k in enumerate(nodes):

        if ii==0:
            df = df_total[df_total["tipo"] == k]
        else:
            df = pd.concat([df, df_total[df_total["tipo"] == k]])

    df.reset_index(inplace=True)
    df.drop(columns="index", inplace=True)
    df["location"] = col

    print("Se ha obtenido un dataframe con shape:", df.shape)

    return df

In [66]:
df_barcelona = tablas(tipo, barcelona, "Barcelona")

Se ha obtenido un dataframe con shape: (5795, 6)


In [67]:
df_madrid = tablas(tipo, madrid, "Madrid")

Se ha obtenido un dataframe con shape: (8545, 6)


In [69]:
df_bilbao = tablas(tipo, bilbao, "Bilbao")

Se ha obtenido un dataframe con shape: (2481, 6)


In [70]:
df_valencia = tablas(tipo, valencia, "Valencia")

Se ha obtenido un dataframe con shape: (1688, 6)


In [72]:
df_malaga = tablas(tipo, malaga, "Málaga")

Se ha obtenido un dataframe con shape: (610, 6)


In [75]:
df_sevilla = tablas(tipo, sevilla, "Sevilla")

Se ha obtenido un dataframe con shape: (1861, 6)


In [83]:
df_locales = pd.concat([df_barcelona,df_madrid,df_bilbao,df_valencia,df_malaga,df_sevilla],axis=0)

df_locales.reset_index(inplace=True)
df_locales.drop(columns="index", inplace=True)
df_locales

,tipo,lat,lon,name,address,location
0,restaurant,41.381687,2.076645,Ateneu de Sant Just Desvern,NO STREET 9999,Barcelona
1,restaurant,41.383643,2.158303,Amaltea,NO STREET 9999,Barcelona
2,restaurant,41.389518,2.162177,Hanibishi,NO STREET 9999,Barcelona
3,restaurant,41.389997,2.131837,Moncho's House,NO STREET 9999,Barcelona
4,restaurant,41.383672,2.180639,Taller de Tapas,l'Argenteria 51,Barcelona
...,...,...,...,...,...,...
20975,theatre,37.385602,-6.003795,Casalá Teatro,NO STREET 9999,Sevilla
20976,theatre,37.383316,-6.000608,Teatro Flamenco Triana,NO STREET 9999,Sevilla
20977,theatre,37.401108,-5.988291,La sala,NO STREET 9999,Sevilla
20978,theatre,37.367891,-5.960394,Teatro Salvador Távora,8 9999,Sevilla


## PRUEBA

In [56]:
df_barcelona

,tipo,lat,lon,name,address
0,restaurant,41.381687,2.076645,Ateneu de Sant Just Desvern,NO STREET 9999
1,restaurant,41.383643,2.158303,Amaltea,NO STREET 9999
2,restaurant,41.389518,2.162177,Hanibishi,NO STREET 9999
3,restaurant,41.389997,2.131837,Moncho's House,NO STREET 9999
4,restaurant,41.383672,2.180639,Taller de Tapas,l'Argenteria 51
...,...,...,...,...,...
4578,theatre,41.369839,2.137810,Sala Sant Medir,NO STREET 9999
4579,theatre,41.370484,2.123253,Auditori la Torrassa,Carrer de Santiago Apòstol 40
4580,theatre,41.378793,2.134733,NO NAME,NO STREET 9999
4581,theatre,41.388456,2.173889,Sala Ars,NO STREET 9999


In [28]:
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
node["amenity"]
  (41.33,2.06,41.46,2.27); 
out;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
data = response.json()
df_bcn = json_to_df(data)
df.head()

,tipo,lat,lon,name,address
0,restaurant,41.381687,2.076645,Ateneu de Sant Just Desvern,NO STREET 9999
1,restaurant,41.383643,2.158303,Amaltea,NO STREET 9999
2,restaurant,41.389518,2.162177,Hanibishi,NO STREET 9999
3,restaurant,41.389997,2.131837,Moncho's House,NO STREET 9999
4,restaurant,41.383672,2.180639,Taller de Tapas,l'Argenteria 51


In [36]:
df_bcn[df_bcn["tipo"]=="restau"]

,tipo,lat,lon,name,address
91,cafe,41.382397,2.074083,Parador,NO STREET 9999
177,cafe,41.380345,2.170768,Mendizabal,NO STREET 9999
188,cafe,41.438684,2.217155,Rin,carrer de Pérez Galdós 29
213,cafe,41.430517,2.172519,The New Orleans Coffee & Tea Company,NO STREET 9999
217,cafe,41.396232,2.189086,Bar Almogávares,NO STREET 9999
...,...,...,...,...,...
25296,cafe,41.376309,2.118485,Bar Cafetería Les Corts,Travessera de les Corts 9999
25297,cafe,41.363198,2.139289,365,NO STREET 9999
25315,cafe,41.397094,2.179024,Vivari,NO STREET 9999
25316,cafe,41.410289,2.137618,Vivari,NO STREET 9999


In [33]:
df_bcn


,tipo,lat,lon,name,address
0,parking_entrance,41.375370,2.152592,NO NAME,NO STREET 9999
1,parking_entrance,41.387344,2.164956,NO NAME,NO STREET 9999
2,pharmacy,41.402512,2.158573,"Farmàcia Mauri Martínez, Emília",Carrer de Ramón y Cajal 24
3,pharmacy,41.403115,2.157597,"Farmàcia Valentines Nart, T - Gelabert Valentines",Carrer de Verdi 7
4,pharmacy,41.403947,2.155336,"Farmàcia Gil Pellicer, F. - Cortés Tena, J.",Carrer d'Astúries 54
...,...,...,...,...,...
25314,drinking_water,41.355736,2.070438,Drinking fountain,NO STREET 9999
25315,cafe,41.397094,2.179024,Vivari,NO STREET 9999
25316,cafe,41.410289,2.137618,Vivari,NO STREET 9999
25317,cafe,41.397786,2.174367,365 Obrador,NO STREET 9999
